# Data 3-4 Pandas Iterations and lambdas

Apply, lambdas, iterrows, itertuples

In this lesson we will start learning how to clean a dataframe data and loop over it

In [14]:
import pandas as pd

checks = pd.read_csv('https://raw.githubusercontent.com/mafudge/datasets/refs/heads/master/dining/check-data.csv')
checks.sample(10)

,check,date,party size,total items on check,total amount of check,gratuity
12,4528,2024-01-02,4,6,$49.98,$7.50
29,4590,2024-05-08,3,5,$220.40,$22.04
11,3693,2024-01-18,10,20,"$1,820.00",$309.40
16,3694,2024-11-03,5,17,"$1,574.37",$173.18
32,1440,2024-11-30,3,8,$589.04,$141.37
34,1368,2024-12-21,10,25,"$2,193.00",$372.81
30,2705,2024-07-08,10,19,$838.85,$671.08
9,2968,2024-12-28,1,3,$122.97,$23.36
3,1957,2024-02-15,2,2,$42.44,$8.91
24,4310,2024-11-30,10,34,"$3,262.30",$913.44


In [15]:
# Note: numbers are not numbers!!!
checks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   check                  50 non-null     int64 
 1   date                   50 non-null     object
 2   party size             50 non-null     int64 
 3   total items on check   50 non-null     int64 
 4   total amount of check  50 non-null     object
 5   gratuity               50 non-null     object
dtypes: int64(3), object(3)
memory usage: 2.5+ KB


## Apply

Apply allows us to execute a function over a Series or the entire DataFrame.


`series.apply(func)` <== call function `func` for every item in the Series

`dataframe.apply(lambda row: func, axis=1)` <== call function `func` for every row in the DataFrame `axis=1` == 

- dataframe → your DataFrame
- apply() → apply a function
- lambda row: → define an anonymous function
- row → represents one entire row at a time (as a Series)
- axis=1 → operate across rows

For each row:

- Pandas passes the row as a Series to the lambda
- The lambda computes a result
- The results are collected into a new Series
- That Series becomes the new column


`dataframe.apply(lambda col: func, axis=0)` <== call function `func` for every row in the DataFrame `axis=0` == col


### Why Apply ?

Apply helps us clean up our data because we can execute non-trivial transformations over our dataframes.

For example, we want to enhance this data by calculating the `price per item` this is defined as:

`total amount of check` / `total items on check`

The problem is `total amount of check` is an `object`, not a `float`. This means we cannot do math on it.


In [4]:
# Type error because of the dollar sign and commas!!!
checks['price_per_item'] = checks['total amount of check'] / checks['total items on check']

TypeError: unsupported operand type(s) for /: 'str' and 'int'

How do we fix this? we write a user-defined function to convert string values like this: `$4,590.45` into floats like this: `4590.45`

In [16]:
def clean_currency(value:str) -> float:
    '''
    This function will take a string value and remove the dollar sign and commas
    and return a float value.
    '''
    return float(value.replace(',', '').replace('$', ''))


# tests
assert clean_currency('$1,000.00') == 1000.00
assert clean_currency('$1,000') == 1000.00
assert clean_currency('1,000') == 1000.00
assert clean_currency('$1000') == 1000.00

In [ ]:
def extract_year(date_in_ymd:str)->int:
    '''
    This function will take a string date an the format 'YYYY-MM-DD'
    and return the year as an integer

    '''
    return int(date_in_ymd.split('-')[0])

assert extract_year("2026-01-01") ==2026

With our function written we can use `apply()` to transform the series.

Remember its a really good idea to **track lineage** when you are building a data pipeline. 

**NEVER** replace columns, always create new ones.

In [20]:
checks['total_amount_of_check_cleaned'] = checks['total amount of check'].apply(clean_currency)
checks['price_per_item'] = checks['total_amount_of_check_cleaned'] / checks['total items on check']
checks.sample(10)

,check,date,party size,total items on check,total amount of check,gratuity,gratuity_cleaned,total_amount_of_check_cleaned,price_per_item
22,1336,2024-08-30,8,28,"$1,199.80",$275.95,275.95,1199.80,42.85
40,2512,2024-03-30,3,12,$181.56,$39.94,39.94,181.56,15.13
33,3842,2024-03-31,6,6,$147.12,$5.88,5.88,147.12,24.52
6,2527,2024-03-27,6,21,$921.48,$55.29,55.29,921.48,43.88
21,4440,2024-06-11,1,3,$168.96,$10.14,10.14,168.96,56.32
11,3693,2024-01-18,10,20,"$1,820.00",$309.40,309.40,1820.00,91.00
38,2341,2024-06-03,7,16,"$1,118.88",$313.29,313.29,1118.88,69.93
9,2968,2024-12-28,1,3,$122.97,$23.36,23.36,122.97,40.99
32,1440,2024-11-30,3,8,$589.04,$141.37,141.37,589.04,73.63
19,3718,2024-10-30,2,5,$464.70,$120.82,120.82,464.70,92.94


## Challenge 3-4-1

#### Modularize our work!

Let's take what we did so far, and create a dataset that would be better prepared for analysis / machine learning.

1. create a module `check_functions.py`
    - add the `clean_currency()` function definition to it.
    - under `if __name__=='__main__':` add the tests
    - run the code to make sure it works.
2. create your challenge file `3-4-1.py`
    - import streamlit, pandas and your clean_currency function
    - load the checks dataset into a dataframe: 
    - clean the `total amount of check` and `gratuity` columns
    - calculate the `price_per_item`  as total amount of check / total items on check
    - calculate the `price_per_person` as total amont of check / party size
    - calculate the `items_per_person` as total items on check / party size
    - calculate the `tip_percentage` as gratuity / total amount of check 
    - display dataframe
    - describe dataframe
    


checks dataset `https://raw.githubusercontent.com/mafudge/datasets/refs/heads/master/dining/check-data.csv`



## Using Row Apply to setup some KPI's 

**KPI** is a key performance indicator. It summarizes larger points of data, so they can be measured over time. For example a letter grade such as an A- is a KPI summary of all your graded efforts to date.

- Let's build some simple KPIs from this data. 
- While KPI's are determined and decided upon from the business decision makers,
- Actionable KPI's are always based on data evidence.

**KPI 1: Whales**

With your help, marketing has decided that whale customers have checks that are:
    - `whale` whales are in the top 75% percentile for both items per person and price per person 
    - `big eaters` are in the top 75% for items per person 
    - `big spenders` are in the top 75% for price per person 

**KPI 2: Tippers**

With your help, marketing has decided that `light` tippers are in the botton 25% of tip percentage and `heavy` tippers are in the top 75% 

### Before we can apply our KPI's we must write the functions!

In [21]:
checks['gratuity_cleaned'] = checks['gratuity'].apply(clean_currency)
checks['price_per_item'] = checks['total_amount_of_check_cleaned'] / checks['total items on check']
checks['price_per_person'] = checks['total_amount_of_check_cleaned'] / checks['party size']
checks['items_per_person'] = checks['total items on check'] / checks['party size']
checks['tip_percentage'] = checks['gratuity_cleaned'] / checks['total_amount_of_check_cleaned']

'''
Determines whether a person is:
A "whale" (top spender and top consumer),
A "big eater" (top consumer only),
A "big spender" (top spender only),
Or none of the above
'''

'\nDetermines whether a person is:\nA "whale" (top spender and top consumer),\nA "big eater" (top consumer only),\nA "big spender" (top spender only),\nOr none of the above\n'

In [29]:
def detect_whale(
        items_per_person:float, 
        price_per_person:float, 
        items_per_person_75th_pctile:float, 
        price_per_person_75_pctile:float) -> str:
    if items_per_person > items_per_person_75th_pctile and price_per_person > price_per_person_75_pctile:
        return 'whale'
    if items_per_person > items_per_person_75th_pctile:
        return 'big eater'
    if price_per_person > price_per_person_75_pctile:
        return 'big spender'
    
    return '' # return an empty string if none of the conditions are met.

#Calculating the 75th Percentile
ppp_75 = checks['price_per_person'].quantile(0.75) # calculate the 75th percentile  of price per person
ipp_75 = checks['items_per_person'].quantile(0.75)
print(ppp_75, ipp_75)

# tests
assert detect_whale(5, 250, 3, 158) == 'whale'
assert detect_whale(5, 100, 3, 158) == 'big eater'
assert detect_whale(1, 250, 3, 158) == 'big spender'
assert detect_whale(1, 100, 3, 158) == ''


# Apply the detect_whale function to each row  in the checks DataFrame
checks['whale'] = checks.apply(
    lambda row: detect_whale(
        row['items_per_person'], 
        row['price_per_person'], 
        ipp_75, 
        ppp_75), 
    axis=1)


'''

What’s happening on line 28 -34?

apply(..., axis=1) → go row by row
Each row is passed into the lambda
The lambda calls detect_whale()
The result (a string) is returned
The results become a new column called "whale"
So each customer gets labeled:
    "whale"
    "big eater"
    "big spender"
    or ""


'''
checks.sample(25)

158.35666666666668 3.0


,check,date,party size,total items on check,total amount of check,gratuity,gratuity_cleaned,total_amount_of_check_cleaned,price_per_item,price_per_person,items_per_person,tip_percentage,whale
4,3010,2024-11-14,6,8,$758.16,$181.96,181.96,758.16,94.77,126.360000,1.333333,0.240002,
41,4210,2024-03-15,7,18,"$1,698.30",$203.80,203.80,1698.30,94.35,242.614286,2.571429,0.120002,big spender
42,1361,2024-11-21,7,14,$65.80,$16.45,16.45,65.80,4.70,9.400000,2.000000,0.250000,
6,2527,2024-03-27,6,21,$921.48,$55.29,55.29,921.48,43.88,153.580000,3.500000,0.060001,big eater
30,2705,2024-07-08,10,19,$838.85,$671.08,671.08,838.85,44.15,83.885000,1.900000,0.800000,
45,4031,2024-07-12,9,13,$129.74,$29.84,29.84,129.74,9.98,14.415556,1.444444,0.229998,
48,4161,2024-06-22,9,28,"$1,385.16",$235.48,235.48,1385.16,49.47,153.906667,3.111111,0.170002,big eater
3,1957,2024-02-15,2,2,$42.44,$8.91,8.91,42.44,21.22,21.220000,1.000000,0.209943,
47,3588,2024-08-20,1,2,$123.46,$6.17,6.17,123.46,61.73,123.460000,2.000000,0.049976,
5,2191,2024-01-06,1,3,$17.85,$1.96,1.96,17.85,5.95,17.850000,3.000000,0.109804,


## Challenge 3-4-2

#### Write and test your KPI's!!!

In module `check_functions.py`
1. copy over the `detect_whale()` function and tests
2. write function `detect_tipper(tip_pct, tip_pcy_75th_pctile, tip_pct_25_pctile)`
    - should return either "light", "heavy" or ""
3. write tests for `detect_tipper()`

in `3-4-2.py`
1. copy the code from `3-4-1.py`
2. Calculate the ntiles using `.quantile()`
3. call the `apply()` function on the row to make new colums `whale` and `tipper`
 

## Looping over Dataframes

If you must for loop over your DataFrames, there are two choices:

- `df.iterrows()` dict-like iteration
- `df.itertuples()` named-tuple like iteration (faster)


Let's do an example where we display the check number, whale and tipper for "heavy tipper" checks.

In [ ]:
## Using the iterrows() method
# printing out the check number and total amount for each check
print("Total Amount of Whale Checks")
for i,row in checks.iterrows():
    if row['whale'] == 'whale':
        print(i, row['check'], row['total_amount_of_check_cleaned'])
        
#If the row is a "whale" check, prints:
#i: the index of the row in the DataFrame.
#row['check']: likely the check identifier.
#row['total_amount_of_check_cleaned']: the amount as a numeric

Total Amount of Whale Checks
16 3694 1574.37
24 4310 3262.3


In [37]:
# Same example with the itertuples() method
print("Total Amount of Whale Checks")
for row in checks.itertuples():
    if row.whale == 'whale':
        print(row.check, row.total_amount_of_check_cleaned)

Total Amount of Whale Checks
3694 1574.37
4310 3262.3


In [38]:
# Of course you don't need a loop to do this:
checks[checks['whale'] == 'whale'][['check', 'total_amount_of_check_cleaned']]

,check,total_amount_of_check_cleaned
16,3694,1574.37
24,4310,3262.30
